In [ ]:
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import col, count, max, min
from pyspark.sql.pandas.functions import PandasUDFType, pandas_udf

# from pyspark.sql import functions as sf

spark = SparkSession.builder.appName("sql2").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/14 23:18:04 WARN Utils: Your hostname, kenans-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.102 instead (on interface en0)
25/10/14 23:18:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/14 23:18:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/14 23:18:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/10/14 23:18:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/10/14 23:18:04 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/10/14 23:18:04 W

In [ ]:
peopleV2 = spark.createDataFrame(
    [
        Row(name="andrew", age=32),
        Row(name="james", age=23),
        Row(name="james", age=4),
        Row(name="ann", age=12),
        Row(name="Alice", age=2),
        Row(name="Alice", age=3),
        Row(name="Alice", age=6),
        Row(name="Bob", age=5),
        Row(name="Bob", age=10),
        Row(name="Bob", age=12),
        Row(name="Cathy", age=7),
        Row(name="Cathy", age=8),
    ]
)

peopleV2.groupby("name").agg(count("*").alias("count")).sort(col("count").desc()).show()

+------+-----+
|  name|count|
+------+-----+
| Alice|    3|
|   Bob|    3|
| james|    2|
| Cathy|    2|
|andrew|    1|
|   ann|    1|
+------+-----+



In [ ]:
peopleV2.groupby("name").agg({"*": "count"}).withColumnRenamed("count(1)", "count").sort("count", ascending=True).show()

+------+-----+
|  name|count|
+------+-----+
|andrew|    1|
|   ann|    1|
| james|    2|
| Cathy|    2|
| Alice|    3|
|   Bob|    3|
+------+-----+



In [ ]:
peopleV2.groupby("name").agg(count("*").alias("count")).sort(col("count").asc()).show()

+------+-----+
|  name|count|
+------+-----+
|andrew|    1|
|   ann|    1|
| james|    2|
| Cathy|    2|
| Alice|    3|
|   Bob|    3|
+------+-----+



In [ ]:
peopleV2.groupby("name").agg(min("age").alias("min_age"), max("age").alias("max_age")).sort(col("min_age").asc()).show()

+------+-------+-------+
|  name|min_age|max_age|
+------+-------+-------+
| Alice|      2|      6|
| james|      4|     23|
|   Bob|      5|     12|
| Cathy|      7|      8|
|   ann|     12|     12|
|andrew|     32|     32|
+------+-------+-------+



In [ ]:
@pandas_udf("int", PandasUDFType.GROUPED_AGG)
def min_udf(v):
    return v.min()


peopleV2.groupby("name").agg(min_udf(peopleV2.age).alias("min_age")).sort("name").show()

/Users/kenanhancer/Documents/projects/python-projects-kenanhancer/python_pyspark_demo/.venv/lib/python3.13/site-packages/pyspark/sql/pandas/functions.py:446: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+------+-------+
|  name|min_age|
+------+-------+
| Alice|      2|
|   Bob|      5|
| Cathy|      7|
|andrew|     32|
|   ann|     12|
| james|      4|
+------+-------+

